# Wikipedia Article Summarization

This class summarizes a Wikipedia page using [Hugging Face's DistilBart text summarization model](https://huggingface.co/sshleifer/distilbart-cnn-12-6) given a valid article.

### How it works/How to use:
1.   Instantiate WikiSummarizer
```
      w = WikiSummarizer()
```
2.   Pass in a URL to the `summary` method of the instantiated object
```
      url = 'https://en.wikipedia.org/wiki/Ww2'
      w.summarize(url)
```

3. Optionally pass in an integer for max_splits. This parameter defines how long the summarization should be. The `distilbart-cnn-12-6` model can only summarize 500 words at a time. By default `max_splits` is 1, so the first 500 words will be summarized. Setting `max_splits` to 2 will summarize and return the first 1000 words, and so on. It may take a while to run anything over the default setting, so be conservative with this parameter.
```
      w.summarize(url, max_splits=2)
```





In [1]:
%%capture
!pip install transformers

In [2]:
import requests
from bs4 import BeautifulSoup
import re
from transformers import pipeline

In [73]:
class WikiSummarizer():
  def __init__(self):
    pass
  
  def __remove_tags(self, t):
    '''
    Removes HTML tags and removes unneeded characters from text.
    '''
    t = str(t)
    t = t.replace('\\', '')
    t = t.replace('\n', '')
    t = re.sub('\[\d+]', '', t)
    t = re.sub('<[^>]+>', '', t)
    return t

  def __retrieve_and_clean_text(self, url):
    '''
    Retrieves text from Wikipedia article, cleans it, and returns the full page as a string.
    '''
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    text = soup.find_all('p')

    return ' '.join(list(map(self.__remove_tags, text)))


  def summarize(self, url, max_splits: int = 1):
    '''
    Summarizes Wikipedia article.
    '''

    # Checks if URL is a string and if URL is valid.
    assert type(url) == str, 'url parameter must be a string.'
    assert 'https://en.wikipedia.org/wiki' in url, 'Must be a valid wikipedia article.'
    

    # Sets the minimum and maximum length of the document.
    MIN_LENGTH = 56
    MAX_LENGTH = 142
    summarized_list, start, end, iter = [], 0, 500, 500

    # Cleans text
    text = self.__retrieve_and_clean_text(url)

    # If the document length is smaller than the max length of the model, the max length is reduced to avoid unnecessary computation.
    doc_length = len(text.split())
    if doc_length < MAX_LENGTH:
      MAX_LENGTH, MIN_LENGTH, end = doc_length, 0, doc_length

    # Instantiates HuggingFace's DistilBart model for text summarization.
    summary_model = pipeline('summarization', 
                             model='sshleifer/distilbart-cnn-12-6',
                             max_length = MAX_LENGTH,
                             min_length = MIN_LENGTH)
    
    # Summarizes text 500 words at a time, depending on max_splits parameter.
    for s in range(0, max_splits):
      summary = summary_model(' '.join(text.split()[start:end]))
      summarized_list.append(summary[0]['summary_text'])
      start = end
      end = end + iter
      # If the end of the article is reached but there are stil more loops to go through.
      if end > len(text.split()):
        break

    # Returns summarized text as a string
    return ' '.join(summarized_list)

In [72]:
w = WikiSummarizer()
url = 'https://en.wikipedia.org/wiki/World_War_II'
w.summarize(url)

" World War II or the Second World War, often abbreviated as WWII or WW2, was a global war that lasted from 1939 to 1945 . It involved the vast majority of the world's countries, including all of the great powers, forming two opposing military alliances: the Allies and the Axis powers . Aircraft played a major role in the conflict, enabling the strategic bombing of population centres and the only two uses of nuclear weapons in war . It resulted in 70 to 85 million fatalities, a majority being civilians ."

In [74]:
w.summarize(url, max_splits=3)

" World War II or the Second World War, often abbreviated as WWII or WW2, was a global war that lasted from 1939 to 1945 . It involved the vast majority of the world's countries, including all of the great powers, forming two opposing military alliances: the Allies and the Axis powers . Aircraft played a major role in the conflict, enabling the strategic bombing of population centres and the only two uses of nuclear weapons in war . It resulted in 70 to 85 million fatalities, a majority being civilians .  World War II changed the political alignment and social structure of the globe . The United Nations was established to foster international co-operation and prevent future conflicts . The Soviet Union and the United States emerged as rival superpowers, setting the stage for the Cold War . The influence of its great powers waned, triggering the decolonisation of Africa and Asia .  The exact date of the war's end is also not universally agreed upon . It was generally accepted at the tim